# Ollama + OpenAI + Python

## 1. Określ nazwę modelu

Jeśli zaimportowałeś inny model niż "phi3:mini", zmień wartość w poniższej komórce. Ta zmienna będzie używana w kodzie w całym notatniku.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Konfiguracja klienta OpenAI

Zazwyczaj klient OpenAI jest używany z OpenAI.com lub Azure OpenAI do interakcji z dużymi modelami językowymi.  
Może być jednak również używany z Ollama, ponieważ Ollama udostępnia zgodny z OpenAI punkt końcowy pod adresem "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Wygeneruj odpowiedź w rozmowie

Teraz możemy użyć OpenAI SDK, aby wygenerować odpowiedź w rozmowie. Ten przykład powinien wygenerować haiku o kotach:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Inżynieria promptów

Pierwsza wiadomość wysyłana do modelu językowego nazywana jest "wiadomością systemową" lub "promptem systemowym" i ustala ogólne instrukcje dla modelu.  
Możesz dostarczyć własny, niestandardowy prompt systemowy, aby nakierować model językowy na generowanie odpowiedzi w inny sposób.  
Zmodyfikuj `SYSTEM_MESSAGE` poniżej, aby model odpowiadał jak Twój ulubiony bohater filmowy lub telewizyjny, albo zainspiruj się innymi promptami systemowymi z [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Po dostosowaniu wiadomości systemowej, podaj pierwsze pytanie użytkownika w `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Przykłady z niewielką ilością danych

Innym sposobem na ukierunkowanie modelu językowego jest dostarczenie "kilku przykładów", czyli sekwencji pytań i odpowiedzi, które pokazują, jak powinien reagować.

Poniższy przykład próbuje nakłonić model językowy do działania jak asystent dydaktyczny, dostarczając kilka przykładów pytań i odpowiedzi, które mógłby udzielić asystent, a następnie zadaje modelowi pytanie, które mógłby zadać student.

Wypróbuj to najpierw, a następnie zmodyfikuj `SYSTEM_MESSAGE`, `EXAMPLES` i `USER_MESSAGE` dla nowego scenariusza.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Generowanie wspomagane wyszukiwaniem

RAG (Retrieval Augmented Generation) to technika umożliwiająca modelowi językowemu udzielanie precyzyjnych odpowiedzi w określonej dziedzinie, poprzez najpierw wyszukiwanie odpowiednich informacji ze źródła wiedzy, a następnie generowanie odpowiedzi na podstawie tych informacji.

Udostępniliśmy lokalny plik CSV zawierający dane o samochodach hybrydowych. Kod poniżej odczytuje plik CSV, wyszukuje dopasowania do pytania użytkownika, a następnie generuje odpowiedź na podstawie znalezionych informacji. Zauważ, że proces ten zajmie więcej czasu niż w poprzednich przykładach, ponieważ wysyłane jest więcej danych do modelu. Jeśli zauważysz, że odpowiedź nadal nie opiera się na danych, możesz spróbować inżynierii systemowej lub innych modeli. Ogólnie rzecz biorąc, RAG jest bardziej skuteczny w przypadku większych modeli lub wersji SLM dostosowanych do konkretnego zadania.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
